In [31]:
import os
import numpy as np
import torch
import torch.nn as nn

DATA_PATH = '/Users/mpekey/Desktop/FlyVideo/Peak_Signal_Data'

### Data

2D Sequence Data after concatenating features

Data Shape: (num_examples, features, time)

In [75]:
true_peak_fn = os.path.join(DATA_PATH, 'true_peak_annotations.npy')
false_peak_fn = os.path.join(DATA_PATH, 'false_peak_annotations.npy')

true_peak_annotations_array = np.load(true_peak_fn)
false_peak_annotations_array = np.load(false_peak_fn, allow_pickle=True)

In [76]:
true_annot_data = true_peak_annotations_array[:,:,:13]
false_annot_data = false_peak_annotations_array[:,:,:13]

true_annot_labels = np.ones((true_annot_data.shape[0],))
false_annot_labels = np.zeros((false_annot_data.shape[0],))

print('True annotations shape:',true_annot_data.shape, 'False annotations shape:', false_annot_data.shape)
print('True labels shape:', true_annot_labels.shape, 'False labels shape:', false_annot_labels.shape)

peaks_data = np.concatenate((true_annot_data, false_annot_data), axis = 0).astype('float64')
peaks_labels = np.concatenate((true_annot_labels, false_annot_labels), axis = 0).astype('float64')
print('Data shape after concatenation:', peaks_data.shape)
print('Label shape after concatenation:', peaks_labels.shape)

True annotations shape: (2878, 60, 13) False annotations shape: (892, 60, 13)
True labels shape: (2878,) False labels shape: (892,)
Data shape after concatenation: (3770, 60, 13)
Label shape after concatenation: (3770,)


In [95]:
class Signal_Dataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.peaks_data = torch.from_numpy(data).float()
        self.peaks_labels = torch.from_numpy(labels).float()

    def __len__(self):
        return len(self.peaks_data)

    def __getitem__(self, idx):
        return self.peaks_data[idx], self.peaks_labels[idx]

In [96]:
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

batch_size = 32

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(Signal_Dataset(peaks_data, peaks_labels),
                                                                         [train_ratio, val_ratio, test_ratio])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

In [97]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers = num_layers, batch_first = True)
        self.fc = nn.Linear(hidden_size, output_size)

        self.hidden_size = hidden_size
        self.num_layers = num_layers

    def forward(self, x):
        # Initialize hidden state with zeros
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        # Initialize cell state
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        output, _ = self.lstm(x, (h_0, c_0))
        # Index hidden state of last time step
        return self.fc(output[:, -1, :]) 

In [111]:
# Define the LSTM model architecture
input_size = 13  # The number of features in the input (assuming each input sequence has 60 features)
hidden_size = 128  # Number of hidden units in the LSTM
output_size = 1  # The number of output classes (binary prediction)
num_layers = 1

num_epochs = 10
learning_rate = 0.001

In [158]:
# Create the LSTM model
model = LSTMModel(input_size, hidden_size, output_size, num_layers)

# Move the model to the appropriate device (e.g., GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss for binary classification
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [159]:
def train_model(model,
                train_loader,
                val_loader,
                optimizer,
                criterion,
                num_epochs):
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        val_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())

            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        with torch.no_grad():
            model.eval()
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs.squeeze(), labels.float())
                val_loss += loss.item()

        # Print the average loss after each epoch
        average_train_loss = train_loss / len(train_loader)
        average_val_loss = val_loss / len(val_loader)

        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {average_train_loss:.4f}, Val Loss: {average_val_loss:.4f}")

    return model, optimizer

In [160]:
def eval_model(model, loader):
    model.eval()
    correct = 0
    total = 0
    test_loss = 0.0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            
            loss = criterion(outputs.squeeze(), labels.float())
            test_loss += loss.item()
            
            predicted = (torch.sigmoid(outputs) >= 0.5).float()

            total += labels.size(0)
            correct += (predicted.squeeze() == labels.float()).sum().item()

    average_test_loss = test_loss / len(loader)
    accuracy = 100 * correct / total
    print(f"Correct Predictions: {correct} among {total} data")
    print(f"Accuracy: {accuracy:.2f}%, Loss: {average_test_loss:.4f}")

In [161]:
model, optimizer = train_model(model,
                               train_loader,
                               val_loader,
                               optimizer,
                               criterion,
                               num_epochs)

Epoch [1/10], Train Loss: 0.5484, Val Loss: 0.5792
Epoch [2/10], Train Loss: 0.5420, Val Loss: 0.5848
Epoch [3/10], Train Loss: 0.5434, Val Loss: 0.5784
Epoch [4/10], Train Loss: 0.5425, Val Loss: 0.5817
Epoch [5/10], Train Loss: 0.5428, Val Loss: 0.5819
Epoch [6/10], Train Loss: 0.5425, Val Loss: 0.5849
Epoch [7/10], Train Loss: 0.5442, Val Loss: 0.5817
Epoch [8/10], Train Loss: 0.5436, Val Loss: 0.5853
Epoch [9/10], Train Loss: 0.5452, Val Loss: 0.5814
Epoch [10/10], Train Loss: 0.5441, Val Loss: 0.5889


In [163]:
eval_model(model, val_loader)

Correct Predictions: 414 among 565 data
Accuracy: 73.27%, Loss: 0.5889
